<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Настройка

xlsx файл

In [1]:
from google.colab import files
filename = files.upload()
filename = next(iter(filename))

Saving russia_regions.xlsx to russia_regions.xlsx


презентация

In [2]:
from google.colab import files
pptx_file = files.upload()
pptx_file = next(iter(pptx_file))

Saving 13.pptx to 13.pptx


#Скрипт

In [3]:
!pip install selenium
!pip install python-pptx
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install pip install excel-formulas-calculator
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [5]:
import openpyxl
from typing import Dict, List, Tuple

from bs4 import BeautifulSoup
import urllib3
from openpyxl.worksheet.worksheet import Worksheet
from nltk.stem.snowball import SnowballStemmer
from datetime import datetime, date, time, timedelta
from openpyxl.formula import Tokenizer
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from copy import copy
from openpyxl.formatting.rule import FormulaRule

import re
from openpyxl.styles import Color, PatternFill, Font, Border
from pptx import Presentation
from efc.interfaces.iopenpyxl import OpenpyxlInterface
import numpy as np
urllib3.disable_warnings()

In [6]:
def get_general_info(info: dict):
    STOP_COVID_URL = 'https://стопкоронавирус.рф/information/'
    wd.get(STOP_COVID_URL)
    raw_rows = wd.find_elements_by_xpath('//h3[@class="cv-stats-virus__item-value"]')
    rows = [row.text for row in raw_rows]
    statistics = [int(''.join(row.strip('\n ').split())) for row in rows]
    useful_statistics = [statistics[2], statistics[4], statistics[0]]
    info.update({'general': useful_statistics})

In [7]:
URL = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json'
REGION_COLUMN = 2
REGION_ROWS_RANGE = (2, 87)
WEEKLY_REPORT_DAY = 7
DAYS_OF_THE_WEEK_MAP = {
    1: 'ПН',
    2: 'ВТ',
    3: 'СР',
    4: 'ЧТ',
    5: 'ПТ',
    6: 'СБ',
    7: 'ВС',
}

def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'автономная', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    stemmer = SnowballStemmer("russian")
    string = ' '.join(stemmer.stem(w) for w in splitted_string if w not in stop_words)
    return string


def fill_сases_sheet(ws: Worksheet, info: dict):
    OVERALL_CASES_FORMULA_ROW = 88

    last_available_date = datetime.strptime(info['dates'][-1], '%Y-%m-%d')
    last_column = ws.max_column
    last_table_date = ws.cell(row=1, column=last_column).value
    new_days_amount = (last_available_date - last_table_date).days
    for i in range(1, new_days_amount + 1):
        ws.cell(row=1, column=(last_column + i)).value = (last_table_date + timedelta(days=i))
        ws.cell(row=1, column=(last_column + i)).number_format = 'DD.MM.YYYY'

        for row in range(*REGION_ROWS_RANGE):
            name = ws.cell(row=row, column=REGION_COLUMN).value
            key = normolize_string(name)
            if key in info['data']:
                ws.cell(row=row, column=(last_column + i)).value = \
                    info['data'][key]['cases'][(i - 1) - new_days_amount][0]
            else:
                raise KeyError(key)

        formula = ws.cell(row=OVERALL_CASES_FORMULA_ROW, column=ws.max_column - 1).value
        last_cell_id = get_column_letter(ws.max_column - 1) + str(OVERALL_CASES_FORMULA_ROW)
        next_cell_id = get_column_letter(ws.max_column) + str(OVERALL_CASES_FORMULA_ROW)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)

    info.update({'new_days_amount': new_days_amount})

import requests
def get_regions_info() -> dict:
    final_region_data = {}

    response: dict = requests.get(URL, verify=False).json()
    regions = response['russia_stat_struct']['data']
    dates = response['russia_stat_struct']['dates']
    for region in regions:
        full_name = regions[region]['info']['name']
        short_name = regions[region]['info']['short_name']
        regions[region]['info'].pop('name', None)
        regions[region]['info'].pop('short_name', None)
        final_region_data.update(
            dict.fromkeys([normolize_string(full_name), normolize_string(short_name)], regions[region]))
    return {'data': final_region_data,
            'dates': dates,
            'last_date': datetime.strptime(dates[-1], '%Y-%m-%d')}


def continue_formula_right(ws: Worksheet, rows: list, column: int):
    for row in rows:
        formula = ws.cell(row=row, column=column - 1).value
        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def  continue_date_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]][3:].index(None) + 4
        except ValueError:
            column = ws.max_column + 1
        last_table_date = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=1)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YYYY'

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_number_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]].index(None) + 1
        except ValueError:
            column = ws.max_column + 1
        last_num = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_GAIN = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(NEW_COLUMN_NUM_GAIN+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 96)), NEW_COLUMN_NUM_GAIN + i)

    info['last_col_in_gain_sheet'] = [c.value for c in ws[1]].index(None)


def daily_region_gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_DAILY_GAIN = ws.max_column + 1

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1, 20, 35])
        continue_formula_right(ws,
                               list(range(2, 19)) + list(range(21, 34)) + list(range(36, 49)),
                               NEW_COLUMN_NUM_DAILY_GAIN + i)


def weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 94)), new_col)
    for row in range(2, 87):
        # left part
        raw_formula = ws.cell(row=row, column=new_col).value
        tokenized_formula = Tokenizer(raw_formula)
        last_col_in_gain_sheet = info['last_col_in_gain_sheet']

        tokenized_formula.items[1].value = re.sub(r'(?<=:\$)\D*',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[1].value)
        tokenized_formula.items[3].value = re.sub(r'(?<=:\$).*(?=\$)',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[3].value)
        new_formula = ''.join(token.value for token in tokenized_formula.items)
        ws.cell(row=row, column=new_col).value = fr'={new_formula}'

        # right part
        ws.cell(row=row, column=new_col+2).value = fr'=MAX($C{row}:${get_column_letter(new_col)}{row})'
        ws.cell(row=row, column=new_col+3).value = fr'={get_column_letter(new_col+2)}{row}-{get_column_letter(new_col)}{row}'
        ws.cell(row=row, column=new_col+4).value = fr'={get_column_letter(new_col)}{row}-{get_column_letter(new_col-1)}{row}'
        ws.cell(row=row, column=new_col+6).value = fr'={get_column_letter(new_col)}{row}/{get_column_letter(new_col-1)}{row}-100%'
        ws.cell(row=row, column=new_col+7).value = fr'={get_column_letter(new_col-1)}{row}/{get_column_letter(new_col-2)}{row}-100%'


    ws.cell(row=87, column=new_col+4).value = fr'={get_column_letter(new_col)}{87}-{get_column_letter(new_col-1)}{87}'


def tpr_weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 88)), new_col)


# {}

# def cases_sheet(wb):
#
#
    # continue_number_right(ws_weekly_gain, )


def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
    for column in columns:

        formula = ws.cell(row=(row - n), column=column).value
        last_cell_id = get_column_letter(column) + str(row - n)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_formula_down(ws: Worksheet, columns: list, row: int):
    continue_formula_n_down(ws, columns, row, 1)


# def get_first_clear_column_in_row(row):
#     return max((c.row for c in column if c.value is not None)) + 1


def get_first_clear_row_in_column(column):
    return max((c.row for c in column if c.value is not None)) + 1

def base_rf_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['B'])

    ws.cell(row=NEW_ROW_NUM, column=1).value = info['last_date']
    ws.cell(row=NEW_ROW_NUM, column=1).number_format = 'DD.MM'
    for i, stat in enumerate(info['general']):
        ws.cell(row=NEW_ROW_NUM, column=2 + i).value = info['general'][i]
        ws.cell(row=NEW_ROW_NUM, column=2 + i)._style = copy(ws.cell(row=NEW_ROW_NUM - 1, column=2 + i)._style)
    continue_formula_down(ws,
                          list(range(5, 20)) + [26] + list(range(28, 36)),
                          NEW_ROW_NUM)

    if info['last_date'].weekday() + 1 == WEEKLY_REPORT_DAY:  # if it's sunday
        continue_formula_n_down(ws, list(range(20, 25))+[27], NEW_ROW_NUM, 7)
        continue_formula_n_down(ws, list(range(20, 23)), NEW_ROW_NUM + 1, 7)
    I_column = [c.value for c in ws['I'][10:]]
    info['day_infections_cell'] = f'I{I_column.index(None)+10}'


def date_week_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['A'])
    ws.cell(row=NEW_ROW_NUM, column=1).value = ws.cell(row=NEW_ROW_NUM - 1, column=1).value + 1
    continue_formula_down(ws, list(range(2, 4)), NEW_ROW_NUM)




def parse_info():
    info = get_regions_info()
    # {dates: [list of available dates], data: {'*REGION_NAME*': {*REGION_INFO*}}}
    get_general_info(info)
    return info


def weekly_region_gain_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1

    continue_number_right(ws, [1, 20])
    continue_formula_right(ws,
                           list(range(2, 17)) + list(range(21, 34)),
                           new_column)

def gain_7day_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 94)), new_column + i)

    for row in range(2, 87):
        ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column+i)}{row}'
        ws.cell(row=row, column=new_column+i+2).value = fr'=RANK({get_column_letter(new_column+i+3)}{row},${get_column_letter(new_column+i+3)}$2:${get_column_letter(new_column+i+3)}$86,0)'
        ws.cell(row=row, column=new_column+i+4).value = fr'={get_column_letter(new_column+i)}{row}-{get_column_letter(new_column+i-7)}{row}'
        ws.cell(row=row, column=new_column+i+6).value = fr'={get_column_letter(new_column+i)}{row}/{get_column_letter(new_column+i-7)}{row}-1'
    ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column + i)}{row}'

    for row in range(92, 97):
        ws.cell(row=row, column=new_column+i+4).value = fr'=VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+3)}$86,2,0)'
        ws.cell(row=row, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{row}/${get_column_letter(new_column+i+4)}$97'
        ws.cell(row=row, column=new_column+i+6).value = fr'==VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+6)}$86,4,0)'
    ws.cell(row=97, column=new_column+i+4).value = fr'={get_column_letter(new_column+i+3)}87'
    ws.cell(row=97, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{97}/${get_column_letter(new_column+i+4)}$97'

def rt_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[2]][3:].index(None) + 4
    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [2])
        continue_formula_right(ws, list(range(3, 89)), new_column + i)

    ws.cell(row=1, column=(new_column+i+5)).value = info['last_date']
    c = get_column_letter(new_column+i+5)
    r = '1'
    cell1 = c+r
    ws.cell(row=1, column=(new_column+i+6)).value = fr'=MATCH({cell1},$A$2:{get_column_letter(new_column+i)}2)-3'

    for row in range(3, 88):
        col = get_column_letter(new_column+i)
        ws.cell(row=row, column=(new_column+i+2)).value = fr'=RANK({col}{row},${col}$3:${col}$87,0)'

        ws.cell(row=row, column=(new_column+i+4)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)<1,1,0)'
        ws.cell(row=row, column=(new_column+i+5)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)>1,1,0)'
        ws.cell(row=row, column=(new_column+i+6)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)=1,1,0)'

    ws.cell(row=88, column=(new_column+i+4)).value = fr'=SUM({get_column_letter(new_column+i+4)}3:{get_column_letter(new_column+i+4)}87)'
    ws.cell(row=88, column=(new_column+i+5)).value = fr'=SUM({get_column_letter(new_column+i+5)}3:{get_column_letter(new_column+i+5)}87)'
    ws.cell(row=88, column=(new_column+i+6)).value = fr'=SUM({get_column_letter(new_column+i+6)}3:{get_column_letter(new_column+i+6)}87)'
    ws.cell(row=89, column=(new_column+i+6)).value = fr'=85-SUM({get_column_letter(new_column+i+4)}88:{get_column_letter(new_column+i+6)}88)'
    info['rt_bigger1_cell'] = f'{get_column_letter(new_column+i+5)}88'


def delta_day_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

        continue_formula_right(ws, list(range(3, 98)), new_column + i)
        cell_ids_formatting = '{}3:{}87'.format(get_column_letter(new_column + i), get_column_letter(new_column + i))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}>0'], fill=bigger_zero_fill))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}<0'], fill=lower_zero_fill))


def msc_sp_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    # bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    # lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)
        continue_formula_right(ws, list(range(3, 98)), new_column + i)


In [8]:

def fill_pptx(wb, info: dict):
    prs = Presentation(pptx_file)
    interface = OpenpyxlInterface(wb=wb, use_cache=True)
    day_infections = interface.calc_cell(info['day_infections_cell'], 'База РФ')
    rt_bigger_1_num = interface.calc_cell(info['rt_bigger1_cell'], 'Rt')
    s1 = "субъекте" if (rt_bigger_1_num%10 == 1) and rt_bigger_1_num != 11 else "субъектах"

    gain = []
    regions = []
    for i in range(2, 87):
        region = interface.calc_cell(f'B{i}', 'Прирост')
        if region not in ['Санкт-Петербург', 'Московская область', 'Москва']:
            gain.append(interface.calc_cell(f'{get_column_letter(info["last_col_in_gain_sheet"])}{i}', 'Прирост'))
            regions.append(region)
    top5_idx = np.array(gain).argsort()[-5:][::-1]
    top5_region = np.array(regions)[top5_idx]
    top5_gain = np.array(gain)[top5_idx]
    s2 = ', '.join([f'{region} ({gain})' for region, gain in zip(top5_region, top5_gain)])
    template = f"{day_infections} новых случаев в России. Ухудшение – в {rt_bigger_1_num} {s1} РФ. Наибольший прирост новых случаев: {s2}."
    # print(template)
    prs.slides[1].shapes[11].text_frame.paragraphs[1].runs[0].text = template
    prs.save('13_out.pptx')

In [9]:
def main(filename: str):
    wb = openpyxl.load_workbook(filename)

    info = parse_info()

    fill_сases_sheet(wb['Случаев'], info)
    gain_sheet(wb['Прирост'], info)
    msc_sp_sheet(wb['МСК и СП'], info)
    rt_sheet(wb['Rt'], info)
    gain_7day_sheet(wb['прирост 7дн'], info)
    delta_day_sheet(wb['дельта за сутки'], info)
    daily_region_gain_sheet(wb['По рег прис (сут)'], info)

    if info['last_date'].weekday()+1 == WEEKLY_REPORT_DAY:
        weekly_gain_sheet(wb['Прирост нед'], info)
        tpr_weekly_gain_sheet(wb['Тпр нед прироста'], info)
        weekly_region_gain_sheet(wb['По рег прис (нед)'], info)
        date_week_sheet(wb['Дата-неделя'], info)

    base_rf_sheet(wb['База РФ'], info)


    fill_pptx(wb, info)
    
    # out_file = filename.split('.')
    # out_file[0] = f'{out_file[0]}_saved'
    # out_file = '.'.join(out_file)
    out_file = 'out.xlsx'
    wb.save(out_file)
    return out_file

# Результат

In [10]:
out_file = main(filename)


In [11]:
files.download('/content/13_out.pptx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
files.download(out_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>